In [10]:
### Import de bibliotecas para manipulação dos dados
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [11]:
# Caminho do arquivo no projeto
pasta_arquivo = '/mnt/c/Users/Isabel/Documents/bootcamp_CDD/avanti-bootcamp-cdd/data/raw/diamonds.csv'

#Leitura do conjunto de dados 
df = pd.read_csv(pasta_arquivo, delimiter=',')
df.head(10)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
5,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48
6,0.24,Very Good,I,VVS1,62.3,57.0,336,3.95,3.98,2.47
7,0.26,Very Good,H,SI1,61.9,55.0,337,4.07,4.11,2.53
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
9,0.23,Very Good,H,VS1,59.4,61.0,338,4.00,4.05,2.39


In [12]:
#print(df.columns)

In [13]:
#Leitura do dicionário
pasta_dicionario = '/mnt/c/Users/Isabel/Documents/bootcamp_CDD/avanti-bootcamp-cdd/data/external/Dicionario_de_dados.csv'
dicionario = pd.read_csv(pasta_dicionario, delimiter=',')
dicionario.head(10)

,Variavel,Tradução,Descrição,Espectro de opções,Tipo,Subtipo
0,carat,Quilate (ct),Peso do diamante em quilates,0.2 ---- 5.01 ct,Quantitativo,Continua
1,cut,Corte,Qualidade do corte,"Fair, Good, Very Good, Premium, Ideal",Qualitativo,Ordinal
2,color,Cor,Cor do diamante,D (best) - J (worst),Qualitativo,Ordinal
3,clarity,Clareza,Clareza do diamante,"1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF ...",Qualitativo,Ordinal
4,depth,Profundidade,Profundidade do diamante,Valor obtido pela equação ( percentual de prof...,Quantitativo,Continua
5,table,Proporção,Proporção base vs topo,43.0 - 95.0,Quantitativo,Continua
6,price,Preço,Valor do diamante em USD,"$326 ---- $18,823",Quantitativo,Discreta
7,x,Comprimento,Comprimento em milímetros (mm),0--10.74 mm,Quantitativo,Continua
8,y,Largura,Largura em milímetros(mm),0--58.9 mm,Quantitativo,Continua
9,z,Altura,Altura em milímetros (mm),0--31.8 mm,Quantitativo,Continua


In [14]:
#print(df.columns)

In [15]:
#Organização da busca pelo tipo de variável

variavel_alvo = 'price'

variavel_ordinal = (
    dicionario
    .query("Subtipo == 'Ordinal' and Variavel != @variavel_alvo")
    .Variavel #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variáveis qualitativo: ordinal")
print(variavel_ordinal)
print(" ")

variavel_quanti_continua = (
    dicionario
    .query("Subtipo == 'Continua' and Variavel != @variavel_alvo")
    .Variavel #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: Contínua")
print(variavel_quanti_continua)
print(" ")

variavel_quanti_discreta = (
    dicionario
    .query("Subtipo == 'Discreta' and Variavel != @variavel_alvo")
    .Variavel #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: discreta")
print(variavel_quanti_discreta)
print(" ")

X = df.drop(columns=[variavel_alvo])
Y = df[variavel_alvo]


Variáveis qualitativo: ordinal
['cut', 'color', 'clarity']
 
Variável quantitativo: Contínua
['carat', 'depth', 'table', 'x', 'y', 'z']
 
Variável quantitativo: discreta
[]
 


In [16]:
processamento_ordinal = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), #Tratamento de dados faltantes com substituição pela MODA
    ("encoding", OrdinalEncoder()), #Codificação de variáveis
     
])

processamento_continuos = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')), #Tratamento de dados faltantes com substituição pela MÉDIA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])
processamento_discreta = Pipeline(steps=[
    ("missing",  KNNImputer()), #Tratamento de dados faltantes com substituição pela MEDIANA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])

In [17]:
pre_processador = ColumnTransformer([
    ("ordinal", processamento_ordinal, variavel_ordinal),
    ("continuo", processamento_continuos, variavel_quanti_continua),
    ("discreta", processamento_discreta, variavel_quanti_discreta)
])

In [18]:
# Modelos e métricas (baseado no código inicial)
modelos = [DummyRegressor(strategy='mean'), LinearRegression(), KNeighborsRegressor(n_neighbors=5), SVR()]
metricas = [
    'neg_mean_absolute_error',
    'neg_mean_squared_error',
    'neg_mean_absolute_percentage_error',
    'r2',
]
monte_carlo = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

# Loop para avaliação dos modelos
resultados_totais = None
for modelo in modelos:
    nome_modelo = modelo.__class__.__name__
    print(f"Rodando para o modelo: {nome_modelo}")
    pipeline = Pipeline(steps=[
        ('preprocessor', pre_processador),
        ('model', modelo),
    ])
    pontuacoes = cross_validate(
        pipeline, X, Y,
        scoring=metricas,
        cv=monte_carlo
    )
    resultados_modelo = pd.DataFrame(pontuacoes)
    resultados_modelo['model'] = nome_modelo
    if resultados_totais is None:
        resultados_totais = resultados_modelo
    else:
        resultados_totais = pd.concat([resultados_totais, resultados_modelo])

# Resumo dos resultados
print(resultados_totais.groupby('model').agg(['mean', 'std']).T)

Rodando para o modelo: DummyRegressor
Rodando para o modelo: LinearRegression
Rodando para o modelo: KNeighborsRegressor
Rodando para o modelo: SVR
model                                         DummyRegressor  \
fit_time                                mean    1.233398e-01   
                                        std     1.336952e-02   
score_time                              mean    3.135464e-02   
                                        std     4.625768e-03   
test_neg_mean_absolute_error            mean   -3.022423e+03   
                                        std     1.702945e+01   
test_neg_mean_squared_error             mean   -1.582750e+07   
                                        std     2.631958e+05   
test_neg_mean_absolute_percentage_error mean   -1.877466e+00   
                                        std     1.333744e-02   
test_r2                                 mean   -5.216110e-05   
                                        std     5.702326e-05   

model              